# Module 6.2 - CNN for Wafer Map Defect Detection

This notebook demonstrates convolutional neural networks for semiconductor wafer map defect classification. We'll work with synthetic wafer maps that simulate real manufacturing patterns.

## Learning Objectives
- Understand spatial pattern recognition in wafer maps
- Implement CNN architectures for defect classification
- Handle class imbalance in manufacturing data
- Apply manufacturing-specific evaluation metrics
- Explore model interpretability with visualization

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import our pipeline
from pathlib import Path
import sys
sys.path.append('.')
from importlib import import_module

# Import pipeline components
try:
    pipeline_module = import_module('6.2-cnn-defect-detection-pipeline')
    print("✓ Pipeline module loaded successfully")
except ImportError as e:
    print(f"⚠ Could not import pipeline: {e}")
    print("Make sure you're running from the module-6 directory")

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Understanding Wafer Map Defect Patterns

Wafer maps show the pass/fail status of dies across a circular silicon wafer. Different defect patterns indicate different manufacturing issues.

In [ ]:
# Generate examples of each defect pattern
patterns = ['normal', 'center', 'edge', 'scratch', 'donut']
pattern_descriptions = {
    'normal': 'Random scattered failures (natural yield loss)',
    'center': 'Central circular defect (spin coating issues)',
    'edge': 'Edge ring pattern (edge bead removal)',
    'scratch': 'Linear defect (mechanical handling)',
    'donut': 'Ring-shaped pattern (temperature gradient)'
}

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
fig.suptitle('Wafer Map Defect Patterns', fontsize=16, fontweight='bold')

for i, pattern in enumerate(patterns):
    wafer = pipeline_module.generate_synthetic_wafer_map(
        pattern=pattern, 
        size=64, 
        noise_level=0.03,
        seed=42
    )
    
    axes[i].imshow(wafer, cmap='RdYlGn', vmin=0, vmax=1)
    axes[i].set_title(f'{pattern.title()}\n{pattern_descriptions[pattern]}', 
                     fontsize=10)
    axes[i].axis('off')
    
    # Add circle to show wafer boundary
    circle = plt.Circle((31.5, 31.5), 31.5, fill=False, color='blue', linewidth=1)
    axes[i].add_patch(circle)

plt.tight_layout()
plt.show()

print("Color coding:")
print("🟢 Green = Passing dies")
print("🟡 Yellow = Marginal")
print("🔴 Red = Failing dies")

## 2. Generate Synthetic Training Dataset

We'll create a synthetic dataset that mimics real wafer map patterns for training our CNN.

In [ ]:
# Generate synthetic dataset
print("Generating synthetic wafer map dataset...")
X_train, y_train, class_names = pipeline_module.generate_synthetic_dataset(
    n_samples=400,  # Modest size for notebook
    image_size=64,
    seed=42
)

print(f"Dataset shape: {X_train.shape}")
print(f"Labels shape: {y_train.shape}")
print(f"Classes: {class_names}")

# Analyze class distribution
unique, counts = np.unique(y_train, return_counts=True)
class_dist = pd.DataFrame({
    'Class': [class_names[i] for i in unique],
    'Count': counts,
    'Percentage': counts / len(y_train) * 100
})

print("\nClass Distribution:")
print(class_dist)

# Visualize class distribution
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.bar(class_dist['Class'], class_dist['Count'])
plt.title('Sample Count by Class')
plt.xlabel('Defect Pattern')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.pie(class_dist['Count'], labels=class_dist['Class'], autopct='%1.1f%%')
plt.title('Class Distribution')

plt.tight_layout()
plt.show()

## 3. Explore Sample Variations

Let's look at multiple examples of each pattern to understand the natural variation in our synthetic data.

In [ ]:
# Show multiple examples of each class
n_examples = 3
fig, axes = plt.subplots(len(class_names), n_examples, figsize=(15, 12))
fig.suptitle('Sample Variations by Defect Pattern', fontsize=16, fontweight='bold')

for class_idx, class_name in enumerate(class_names):
    # Find samples of this class
    class_mask = y_train == class_idx
    class_samples = X_train[class_mask]
    
    for example_idx in range(n_examples):
        if example_idx < len(class_samples):
            sample = class_samples[example_idx]
            axes[class_idx, example_idx].imshow(sample, cmap='RdYlGn', vmin=0, vmax=1)
        else:
            axes[class_idx, example_idx].text(0.5, 0.5, 'No data', 
                                             ha='center', va='center', 
                                             transform=axes[class_idx, example_idx].transAxes)
        
        axes[class_idx, example_idx].axis('off')
        
        if example_idx == 0:
            axes[class_idx, example_idx].set_ylabel(class_name.title(), 
                                                   fontsize=12, fontweight='bold')
        
        if class_idx == 0:
            axes[class_idx, example_idx].set_title(f'Example {example_idx + 1}')

plt.tight_layout()
plt.show()

## 4. Train CNN Model

Now let's train our CNN model on the synthetic data. We'll use the production pipeline we created.

In [ ]:
# Create and train CNN pipeline
print("Training CNN model...")
print(f"PyTorch available: {pipeline_module.HAS_TORCH}")

# Initialize pipeline
cnn_pipeline = pipeline_module.CNNDefectPipeline(
    model_type='simple_cnn',
    num_classes=len(class_names),
    epochs=10,  # More epochs for better learning
    batch_size=32,
    learning_rate=0.001
)

# Train the model
cnn_pipeline.fit(X_train, y_train, class_names)

print("✓ Model training completed")
print(f"Model type: {cnn_pipeline.model_type}")
print(f"Backend: {'PyTorch' if pipeline_module.HAS_TORCH else 'sklearn'}")

## 5. Evaluate Model Performance

Let's evaluate our trained model using both standard ML metrics and manufacturing-specific metrics.

In [ ]:
# Evaluate model performance
print("Evaluating model performance...")
metrics = cnn_pipeline.evaluate(X_train, y_train)

print("\n📊 Performance Metrics:")
print("=" * 40)
print(f"Accuracy:           {metrics['accuracy']:.3f}")
print(f"F1-Score (Macro):   {metrics['f1_macro']:.3f}")
print(f"F1-Score (Weighted): {metrics['f1_weighted']:.3f}")
print(f"ROC-AUC (OvR):      {metrics['roc_auc_ovr']:.3f}")
print(f"PR-AUC (Macro):     {metrics['pr_auc_macro']:.3f}")
print("\n🏭 Manufacturing Metrics:")
print("=" * 40)
print(f"PWS (Prediction Within Spec): {metrics['pws']:.1f}%")
print(f"Estimated Loss:     {metrics['estimated_loss']:.3f}")

# Visualize metrics
metric_names = ['Accuracy', 'F1-Macro', 'F1-Weighted', 'ROC-AUC', 'PR-AUC']
metric_values = [metrics['accuracy'], metrics['f1_macro'], 
                metrics['f1_weighted'], metrics['roc_auc_ovr'], 
                metrics['pr_auc_macro']]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
bars = plt.bar(metric_names, metric_values, 
              color=['skyblue', 'lightgreen', 'lightcoral', 'gold', 'plum'])
plt.title('Model Performance Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.xticks(rotation=45)

# Add value labels on bars
for bar, value in zip(bars, metric_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.3f}', ha='center', va='bottom')

plt.subplot(1, 2, 2)
manufacturing_metrics = ['PWS (%)', 'Est. Loss']
manufacturing_values = [metrics['pws'], metrics['estimated_loss'] * 100]  # Scale loss for visualization
plt.bar(manufacturing_metrics, manufacturing_values, color=['orange', 'red'])
plt.title('Manufacturing-Specific Metrics')
plt.ylabel('Value')

plt.tight_layout()
plt.show()

## 6. Detailed Prediction Analysis

Let's analyze the model's predictions in detail to understand its behavior.

In [ ]:
# Generate predictions and probabilities
predictions = cnn_pipeline.predict(X_train)
probabilities = cnn_pipeline.predict_proba(X_train)

# Convert predictions back to numeric for analysis
pred_numeric = cnn_pipeline.label_encoder.transform(predictions)

# Create confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_train, pred_numeric)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot confusion matrix
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (Counts)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.subplot(1, 3, 2)
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (Normalized)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# Plot prediction confidence distribution
plt.subplot(1, 3, 3)
max_probs = np.max(probabilities, axis=1)
plt.hist(max_probs, bins=20, alpha=0.7, edgecolor='black')
plt.title('Prediction Confidence Distribution')
plt.xlabel('Maximum Probability')
plt.ylabel('Frequency')
plt.axvline(np.mean(max_probs), color='red', linestyle='--', 
           label=f'Mean: {np.mean(max_probs):.3f}')
plt.legend()

plt.tight_layout()
plt.show()

# Print detailed classification report
print("\n📋 Detailed Classification Report:")
print("=" * 50)
print(classification_report(y_train, pred_numeric, target_names=class_names))

## 7. Analyze Misclassifications

Let's examine cases where the model made incorrect predictions to understand its limitations.

In [ ]:
# Find misclassified samples
misclassified_mask = y_train != pred_numeric
misclassified_indices = np.where(misclassified_mask)[0]

print(f"Total misclassifications: {len(misclassified_indices)} / {len(y_train)} ({100*len(misclassified_indices)/len(y_train):.1f}%)")

if len(misclassified_indices) > 0:
    # Show some misclassified examples
    n_show = min(8, len(misclassified_indices))
    show_indices = misclassified_indices[:n_show]
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle('Misclassified Examples', fontsize=16, fontweight='bold')
    
    for i, idx in enumerate(show_indices):
        row, col = i // 4, i % 4
        
        true_label = class_names[y_train[idx]]
        pred_label = class_names[pred_numeric[idx]]
        confidence = np.max(probabilities[idx])
        
        axes[row, col].imshow(X_train[idx], cmap='RdYlGn', vmin=0, vmax=1)
        axes[row, col].set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}', 
                                fontsize=10)
        axes[row, col].axis('off')
    
    # Hide unused subplots
    for i in range(n_show, 8):
        row, col = i // 4, i % 4
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Analyze misclassification patterns
    misclass_true = y_train[misclassified_mask]
    misclass_pred = pred_numeric[misclassified_mask]
    
    print("\nMost common misclassification patterns:")
    for true_class in range(len(class_names)):
        for pred_class in range(len(class_names)):
            if true_class != pred_class:
                count = np.sum((misclass_true == true_class) & (misclass_pred == pred_class))
                if count > 0:
                    print(f"  {class_names[true_class]} → {class_names[pred_class]}: {count} cases")
else:
    print("🎉 Perfect classification! No misclassifications found.")

## 8. Test on New Synthetic Data

Let's test our trained model on a fresh set of synthetic data to simulate real-world deployment.

In [ ]:
# Generate test data with different random seed
print("Generating fresh test data...")
X_test, y_test, _ = pipeline_module.generate_synthetic_dataset(
    n_samples=100,  # Smaller test set
    image_size=64,
    seed=123  # Different seed for true out-of-sample testing
)

print(f"Test dataset shape: {X_test.shape}")

# Evaluate on test data
test_metrics = cnn_pipeline.evaluate(X_test, y_test)

print("\n🧪 Test Set Performance:")
print("=" * 30)
print(f"Accuracy:     {test_metrics['accuracy']:.3f}")
print(f"F1-Macro:     {test_metrics['f1_macro']:.3f}")
print(f"PWS:          {test_metrics['pws']:.1f}%")
print(f"Est. Loss:    {test_metrics['estimated_loss']:.3f}")

# Compare train vs test performance
comparison_metrics = ['accuracy', 'f1_macro', 'pws']
train_values = [metrics[m] for m in comparison_metrics]
test_values = [test_metrics[m] for m in comparison_metrics]

x = np.arange(len(comparison_metrics))
width = 0.35

plt.figure(figsize=(10, 6))
plt.bar(x - width/2, train_values, width, label='Training', alpha=0.8)
plt.bar(x + width/2, test_values, width, label='Test', alpha=0.8)

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Training vs Test Performance')
plt.xticks(x, [m.replace('_', ' ').title() for m in comparison_metrics])
plt.legend()
plt.ylim(0, 1)

# Add value labels
for i, (train_val, test_val) in enumerate(zip(train_values, test_values)):
    plt.text(i - width/2, train_val + 0.01, f'{train_val:.3f}', 
             ha='center', va='bottom', fontsize=9)
    plt.text(i + width/2, test_val + 0.01, f'{test_val:.3f}', 
             ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Check for overfitting
accuracy_diff = metrics['accuracy'] - test_metrics['accuracy']
print(f"\n📈 Overfitting Analysis:")
print(f"Accuracy difference (train - test): {accuracy_diff:.3f}")
if accuracy_diff > 0.1:
    print("⚠️  Potential overfitting detected (>10% accuracy drop)")
elif accuracy_diff > 0.05:
    print("🟡 Moderate overfitting (5-10% accuracy drop)")
else:
    print("✅ Good generalization (minimal overfitting)")

## 9. Model Persistence and Loading

Demonstrate saving and loading the trained model for production use.

In [ ]:
# Save the trained model
model_path = Path('cnn_wafer_model.joblib')
print(f"Saving model to {model_path}...")

cnn_pipeline.save(model_path)
print(f"✓ Model saved successfully")
print(f"File size: {model_path.stat().st_size / 1024:.1f} KB")

# Load the model
print("\nLoading saved model...")
loaded_pipeline = pipeline_module.CNNDefectPipeline.load(model_path)
print("✓ Model loaded successfully")

# Verify the loaded model works
sample_wafer = X_test[:1]  # Take first test sample
original_pred = cnn_pipeline.predict(sample_wafer)
loaded_pred = loaded_pipeline.predict(sample_wafer)

print(f"\n🔍 Verification:")
print(f"Original model prediction: {original_pred[0]}")
print(f"Loaded model prediction:   {loaded_pred[0]}")
print(f"Predictions match: {original_pred[0] == loaded_pred[0]}")

# Show model metadata
print(f"\n📋 Model Metadata:")
print(f"Model type: {loaded_pipeline.metadata.model_type}")
print(f"Training date: {loaded_pipeline.metadata.trained_at}")
print(f"Input shape: {loaded_pipeline.metadata.input_shape}")
print(f"Classes: {loaded_pipeline.metadata.class_names}")
print(f"PyTorch available: {loaded_pipeline.metadata.pytorch_available}")

## 10. Interactive Prediction Demo

Let's create an interactive demonstration where we can generate specific patterns and see predictions.

In [ ]:
# Interactive prediction demonstration
def predict_and_visualize(pattern, size=64, noise_level=0.05):
    """Generate a wafer map and predict its pattern"""
    # Generate wafer map
    wafer = pipeline_module.generate_synthetic_wafer_map(
        pattern=pattern, size=size, noise_level=noise_level
    )
    
    # Make prediction
    wafer_batch = wafer.reshape(1, size, size)
    prediction = loaded_pipeline.predict(wafer_batch)[0]
    probabilities = loaded_pipeline.predict_proba(wafer_batch)[0]
    
    # Visualize results
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Show wafer map
    axes[0].imshow(wafer, cmap='RdYlGn', vmin=0, vmax=1)
    axes[0].set_title(f'Input Wafer Map\n(True pattern: {pattern})', fontweight='bold')
    axes[0].axis('off')
    
    # Show prediction probabilities
    bars = axes[1].bar(class_names, probabilities, 
                      color=['lightcoral' if cls == prediction else 'lightblue' 
                            for cls in class_names])
    axes[1].set_title(f'Prediction Probabilities\nPredicted: {prediction}', fontweight='bold')
    axes[1].set_ylabel('Probability')
    axes[1].set_ylim(0, 1)
    axes[1].tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, prob in zip(bars, probabilities):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'{prob:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Print results
    confidence = np.max(probabilities)
    is_correct = pattern == prediction
    
    print(f"🎯 True pattern: {pattern}")
    print(f"🤖 Predicted: {prediction}")
    print(f"📊 Confidence: {confidence:.3f}")
    print(f"✅ Correct: {is_correct}")
    
    return is_correct, confidence

# Test on each pattern type
print("🧪 Testing model on each defect pattern:")
print("=" * 50)

results = []
for pattern in patterns:
    print(f"\nTesting {pattern} pattern:")
    is_correct, confidence = predict_and_visualize(pattern)
    results.append((pattern, is_correct, confidence))

# Summary
correct_count = sum(1 for _, correct, _ in results if correct)
avg_confidence = np.mean([conf for _, _, conf in results])

print(f"\n📈 Summary:")
print(f"Correct predictions: {correct_count}/{len(patterns)} ({100*correct_count/len(patterns):.1f}%)")
print(f"Average confidence: {avg_confidence:.3f}")

## 11. Manufacturing Insights and Recommendations

Let's analyze what we've learned and provide actionable insights for semiconductor manufacturing.

In [ ]:
# Analyze model performance by defect type
test_predictions = loaded_pipeline.predict(X_test)
test_pred_numeric = loaded_pipeline.label_encoder.transform(test_predictions)
test_probabilities = loaded_pipeline.predict_proba(X_test)

# Calculate per-class performance
from sklearn.metrics import precision_score, recall_score, f1_score

per_class_metrics = []
for i, class_name in enumerate(class_names):
    class_mask = y_test == i
    if np.sum(class_mask) > 0:  # Only if we have samples of this class
        precision = precision_score(y_test == i, test_pred_numeric == i)
        recall = recall_score(y_test == i, test_pred_numeric == i)
        f1 = f1_score(y_test == i, test_pred_numeric == i)
        
        # Average confidence for this class
        class_confidences = np.max(test_probabilities[class_mask], axis=1)
        avg_confidence = np.mean(class_confidences)
        
        per_class_metrics.append({
            'Class': class_name,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'Avg_Confidence': avg_confidence,
            'Sample_Count': np.sum(class_mask)
        })

# Create performance DataFrame
performance_df = pd.DataFrame(per_class_metrics)
print("📊 Per-Class Performance Analysis:")
print("=" * 60)
print(performance_df.round(3))

# Visualize per-class performance
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Per-Class Performance Analysis', fontsize=16, fontweight='bold')

# Precision by class
axes[0, 0].bar(performance_df['Class'], performance_df['Precision'], color='skyblue')
axes[0, 0].set_title('Precision by Defect Type')
axes[0, 0].set_ylabel('Precision')
axes[0, 0].set_ylim(0, 1)
axes[0, 0].tick_params(axis='x', rotation=45)

# Recall by class
axes[0, 1].bar(performance_df['Class'], performance_df['Recall'], color='lightgreen')
axes[0, 1].set_title('Recall by Defect Type')
axes[0, 1].set_ylabel('Recall')
axes[0, 1].set_ylim(0, 1)
axes[0, 1].tick_params(axis='x', rotation=45)

# F1-Score by class
axes[1, 0].bar(performance_df['Class'], performance_df['F1-Score'], color='lightcoral')
axes[1, 0].set_title('F1-Score by Defect Type')
axes[1, 0].set_ylabel('F1-Score')
axes[1, 0].set_ylim(0, 1)
axes[1, 0].tick_params(axis='x', rotation=45)

# Confidence by class
axes[1, 1].bar(performance_df['Class'], performance_df['Avg_Confidence'], color='gold')
axes[1, 1].set_title('Average Confidence by Defect Type')
axes[1, 1].set_ylabel('Average Confidence')
axes[1, 1].set_ylim(0, 1)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Manufacturing recommendations
print("\n🏭 Manufacturing Recommendations:")
print("=" * 50)

# Find best and worst performing classes
best_f1_class = performance_df.loc[performance_df['F1-Score'].idxmax()]
worst_f1_class = performance_df.loc[performance_df['F1-Score'].idxmin()]

print(f"✅ Best performance: {best_f1_class['Class']} (F1: {best_f1_class['F1-Score']:.3f})")
print(f"⚠️  Needs improvement: {worst_f1_class['Class']} (F1: {worst_f1_class['F1-Score']:.3f})")

# Low confidence warnings
low_confidence_threshold = 0.8
low_conf_classes = performance_df[performance_df['Avg_Confidence'] < low_confidence_threshold]
if len(low_conf_classes) > 0:
    print(f"\n🟡 Low confidence classes (< {low_confidence_threshold}):")
    for _, row in low_conf_classes.iterrows():
        print(f"   - {row['Class']}: {row['Avg_Confidence']:.3f}")
    print("   → Consider collecting more training data for these patterns")

# Recall warnings (missed defects are costly)
low_recall_threshold = 0.8
low_recall_classes = performance_df[performance_df['Recall'] < low_recall_threshold]
if len(low_recall_classes) > 0:
    print(f"\n🔴 Low recall classes (< {low_recall_threshold}):")
    for _, row in low_recall_classes.iterrows():
        print(f"   - {row['Class']}: {row['Recall']:.3f}")
    print("   → High risk of missing critical defects - priority for improvement")

print(f"\n💡 Next Steps:")
print(f"   1. Deploy model for real-time wafer screening")
print(f"   2. Implement confidence-based human review (< 0.8 confidence)")
print(f"   3. Collect more data for underperforming classes")
print(f"   4. Monitor model performance over time for drift")
print(f"   5. Consider ensemble methods for critical applications")

## 12. Cleanup and Summary

Clean up temporary files and summarize what we've accomplished.

In [ ]:
# Clean up saved model file
if model_path.exists():
    model_path.unlink()
    print(f"🗑️  Cleaned up temporary model file: {model_path}")

# Check for PyTorch model file
pytorch_model_path = model_path.with_suffix('.pth')
if pytorch_model_path.exists():
    pytorch_model_path.unlink()
    print(f"🗑️  Cleaned up PyTorch model file: {pytorch_model_path}")

print("\n🎉 Module 6.2 Complete - CNN Wafer Defect Detection")
print("=" * 60)
print("\n📚 What we accomplished:")
print("   ✅ Generated synthetic wafer map dataset with 5 defect patterns")
print("   ✅ Trained CNN model for spatial pattern recognition")
print("   ✅ Evaluated performance with manufacturing-specific metrics")
print("   ✅ Analyzed misclassifications and model behavior")
print("   ✅ Demonstrated model persistence and loading")
print("   ✅ Provided actionable manufacturing insights")

print("\n🔧 Key skills learned:")
print("   • Spatial pattern recognition in manufacturing data")
print("   • CNN architecture design for small datasets")
print("   • Class imbalance handling techniques")
print("   • Manufacturing-specific evaluation metrics")
print("   • Model interpretability and confidence analysis")

print("\n🚀 Ready for production:")
print("   • Use the CLI pipeline for batch processing")
print("   • Implement confidence-based human review")
print("   • Monitor for data drift and model performance")
print("   • Scale with real WM-811K dataset when available")

print("\n🎯 Next modules:")
print("   • Module 6.3: Advanced CNN architectures")
print("   • Module 7.1: Multi-scale analysis")
print("   • Module 8.1: Model explainability (Grad-CAM)")
print("   • Module 9.1: Production deployment")